In [1]:
import sys

sys.path.append("../../")

%load_ext autoreload
%autoreload 2

In [62]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from ast import literal_eval
from pathlib import Path
from functools import reduce
from tqdm import tqdm_notebook

from src.utils import find_meta_category
from src.feature_extractor import sample_feature_combinations

## Download prerequisite files

Fetch all the results and feature values


In [64]:
skip_download = True
if not skip_download:
# You can get the experiments file here: 01J6KF3JRCATRJQ9CPJTRV5VBM (https://beaker.org/ds/01J6KF3JRCATRJQ9CPJTRV5VBM/details)
    !echo "Fetching experiments list..."
    !beaker dataset fetch 01J6KF3JRCATRJQ9CPJTRV5VBM --prefix experiments.txt -q
    !echo "Fetching extracted features..."
    !mkdir -p features/
    !beaker dataset fetch 01J6KF3JRCATRJQ9CPJTRV5VBM --prefix features/ -q
    #!beaker dataset fetch 01J6KFVCRCTYHCZDR0XNK0G9HT --prefix features/
    !echo "Fetching helpsteer2 dataset..."
    !beaker dataset fetch 01J6KBM2VCM9EQ7MER26VBXCCM
    !echo "Fetching extracted subsets... (this will take ~10 minutes)"
    !beaker dataset fetch 01J6KF3JRCATRJQ9CPJTRV5VBM --prefix data/ -q
    
!echo "Collating all evaluation results"
%run ../../scripts/fetch_evals_rewardbench.py --output_file results.csv --gpt4_threshold_score 0.658 --experiment_prefix rm-eval-helpsteer2 --experiments_file experiments.txt

Fetching experiments list...
Fetching extracted features...
mkdir: features/: File exists
Fetching helpsteer2 dataset...
Files: 0          ⠋  
Bytes: 0 B        ⠋  
Files: 1          ⠙  
Bytes: 70.58 MiB  ⠙  
Files: 1          ✔  
Bytes: 70.58 MiB  ✔  
Completed in 200ms: 306 MiB/s, 4 files/s
Fetching extracted subsets...
Collating all evaluation results
2024-09-04 10:55:06 - INFO - root - Logged-in as ljm (ljm@allenai.org)
2024-09-04 10:55:07 - INFO - root - Found 291 experiments that match 'rm-eval-helpsteer2'


100%|██████████| 291/291 [00:38<00:00,  7.59it/s]

2024-09-04 10:55:46 - INFO - root - Computing category scores...
2024-09-04 10:55:46 - INFO - root - Deriving features from the experiments file: experiments.txt
2024-09-04 10:55:46 - INFO - root - Will attempt merge via feature hash
2024-09-04 10:55:46 - INFO - root - Creating labels in column 'label' with GPT-4 threshold '0.658'
2024-09-04 10:55:46 - INFO - root - Saving 128 results to results.csv
2024-09-04 10:55:46 - INFO - root - Saved on results.csv


In [42]:
!ls data | wc -l

     253


Collate feature set for all instances


In [43]:
LEXICAL_FEATS_PATH = Path("features")
DATASET_PATH = Path("helpsteer2_human_vs_gpt4_weighted_for_llama.jsonl")


def get_dataset_features(
    feature_path=LEXICAL_FEATS_PATH, dataset_path=DATASET_PATH
) -> "pd.DataFrame":
    lexical_features = [
        "rouge",
        "bertscore",
        "bertscore_length",
        "entity_sim",
        "cosine_sim",
        "prompt_len",
        "len_longer",
        "len_shorter",
        "token_len_difference",
    ]
    lexical_feature_files = [
        file
        for file in feature_path.glob("*.jsonl")
        if any(file.stem in feat for feat in lexical_features)
    ]
    lexical_feats_df = reduce(
        lambda left, right: left.merge(
            right, on=["id", "prompt", "completion_a", "completion_b"], how="outer"
        ),
        [pd.read_json(file, lines=True) for file in lexical_feature_files],
    )

    df = pd.read_json(dataset_path, lines=True).rename(columns={"prompt_hash": "id"})
    finaldf = df.merge(lexical_feats_df, how="left", on="id").drop(
        columns=["prompt", "completion_a", "completion_b"]
    )

    # Hacky way for token_len_difference
    finaldf = finaldf.rename(columns={"token_len_diff": "token_len_difference"})
    return finaldf

In [48]:
results_df = pd.read_csv("results.csv").dropna()
features_df = get_dataset_features()
features_df.to_csv("helpsteer2_all_features.csv", index=False)
# print(len(results_df)), print(len(features_df))

This dataframe contains the features for instances in the dataset


In [46]:
features_df.head()

,id,text,response_a,response_b,pref_human,pref_gpt4,rating_human,rating_gpt4,completions,subject_of_expertise,...,format_constraints,cosine_sim,token_len_difference,rouge,bertscore,len_longer,prompt_len,bertscore_length,len_shorter,entity_sim
0,000be890ad02c46a3554c77f098f151f,In what ways can researching something be help...,Engaging in research is a valuable endeavor th...,There are several reasons why doing research c...,B-is-clearly-better,A-is-clearly-better,"[2.83, 6.28]","[6.09, 5.98]",[Engaging in research is a valuable endeavor t...,"[Education, Psychology]",...,"[#words <= 200, style = formal]",0.709485,166,0.328704,0.799727,376,25,0.435609,210,1.000000
1,001458ba4f2925f2bc6ae9d8d197d543,Write an inspiring song about alena wicker for...,"Here's a song inspired by Alena Wicker, a youn...",Here's an inspiring song about Alena Wicker fo...,B-is-clearly-better,B-is-clearly-better,"[3.73, 4.77]","[1.59, 2.25]","[Here's a song inspired by Alena Wicker, a you...",[Performing arts],...,"[content = sone, style = inspiring, include: t...",0.844983,61,0.463866,0.842092,503,44,0.724686,442,0.333333
2,0014db4cd30cad106320d9043f45d607,Hi! I am a Hypnotist and transformational / mi...,They want a solution to their problems. They w...,They want a solution to lead a more fulfilling...,B-is-clearly-better,A-is-clearly-better,"[4.77, 5.95]","[5.87, 5.62]",[They want a solution to their problems. They ...,"[Business, Psychology]",...,[],0.508719,424,0.150259,0.772617,480,209,0.000456,56,0.000000
3,003af4a043a0cb2186bc448a290d35f6,Australian Defence Force Military Police membe...,"Yes, members of the Australian Defence Force M...","Yes, members of the Australian Defence Force M...",Tie,B-is-clearly-better,"[6.28, 6.28]","[5.62, 6.09]","[Yes, members of the Australian Defence Force ...","[Military sciences, Law]",...,[],0.872330,121,0.510121,0.881362,217,30,0.268603,96,0.500000
4,003dd8aa40b027ae755e47917c89c0a3,Heather: I want to attend the music festival t...,It is unclear if all girls have enough money t...,Yes\n\nBoth Heather and Kara state that they h...,A-is-clearly-better,A-is-clearly-better,"[6.14, 3.02]","[5.13, 0.64]",[It is unclear if all girls have enough money ...,"[Performing arts, Economics]",...,[],0.839198,16,0.474227,0.866347,68,107,0.671132,52,0.600000


## Get proportion of instances that fulfill the conditions

1. For each row, get features that were activated
2. Then for each activated feature, we get the proportion by looking at the feature dataframe.
3. The proportion is computed as: `number_of_instance_that_fulfill_a_single_condition` / `total_number_of_instances`


In [45]:
# Inspect nan columns
rows_with_nan = features_df[features_df.isna().any(axis=1)]
nan_columns = rows_with_nan.columns[rows_with_nan.isna().any()]
df_nan_columns = rows_with_nan[nan_columns]
df_nan_columns

,expertise_level,format_constraints
289,None,[]
1317,expert domain knowledge,None
4613,basic domain knowledge,None
4734,general public,None


So what you're going to do instead, is to take the binary_cols, and then for each element of that binary_cols, you compute the "weight"


In [50]:
def compute_instances(feat: str, features_df: "pd.DataFrame") -> float:
    """Compute the ratio of instances that fulfill a given feature 'feat' vs. the total dataset 'len(features_df)'"""
    total = len(features_df)
    lexical_features = [
        "rouge",
        "bertscore",
        "bertscore_length",
        "entity_sim",
        "cosine_sim",
        "prompt_len",
        "len_longer",
        "len_shorter",
        "token_len_difference",
    ]

    if feat.split("__")[0] in lexical_features:
        feat_name, value = feat.split("__")
        min_val_str, max_val_str = value.split("|")
        min_val, max_val = float(min_val_str.split("=")[1]), float(
            max_val_str.split("=")[1]
        )
        return features_df[feat_name].between(min_val, max_val).mean()
    else:
        # Parse the feature
        feat_name, value = feat.split("=")
        meta_category = find_meta_category(feat_name)
        if meta_category == "scalar":
            v = value.replace("_", " ")
            return features_df[feat_name].value_counts().get(v) / total
        elif meta_category == "closed_set":
            v = value.replace("_", " ")
            list_of_values = features_df[feat_name].tolist()
            return sum([1 if v in listval else 0 for listval in list_of_values]) / total
        elif meta_category == "open_set":
            list_of_values = features_df[feat_name].tolist()
            return sum([1 if listval else 0 for listval in list_of_values]) / total

        return find_meta_category(feat_name)


# feats = results_df.columns[results_df.isin([0, 1]).all()]  # get binary columns
# feat_map = {
#    feat: compute_instances(feat, features_df) for feat in feats if feat != "label"
# }

# ratio_df = results_df.apply(
#    lambda row: row.map(lambda x: feat_map.get(row.name, 1) if x == 1 else x)
# )

For each result, we get the `hash`, find the extracted subset (because they were randomly-sampled) from `data`, and compute the ratio from there.


In [76]:
import re


def extract_hash(string):
    match = re.search(r"FEATS_(.*?)_SWAPS", string)
    return match.group(1) if match else None


result_hashes = results_df["hash"].to_list()
subsets = {extract_hash(str(file)): file for file in Path("data").glob("*.jsonl")}
feats = results_df.columns[results_df.isin([0, 1]).all()]  # get binary collumns

hash_ratios = {}
for result_hash in tqdm_notebook(result_hashes):
    if result_hash in subsets:
        sampled_features_df = pd.read_json(subsets[result_hash], lines=True)
        sampled_features_df["id"] = sampled_features_df["prompt"].apply(
            lambda x: hashlib.md5(x.encode("utf-8")).hexdigest()
        )
        # Get the features from features_df based on the existing prompt_hashes in sampled_features_df
        sdf = features_df[features_df["id"].isin(sampled_features_df["id"].to_list())]
        hash_ratios[result_hash] = {
            feat: compute_instances(feat, sdf) for feat in feats if feat != "label"
        }


def replace_values(row):
    feat_map = hash_ratios.get(row["hash"], {})
    for col in feat_map:
        if row[col] == 1 and col in feat_map:
            row[col] = feat_map[col]
    return row


ratio_df = results_df.apply(replace_values, axis=1)

/var/folders/hx/gk2rs0792pn5p8hkj4nkhdm80000gp/T/ipykernel_24031/1978306616.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for result_hash in tqdm_notebook(result_hashes):


  0%|          | 0/127 [00:00<?, ?it/s]

## Regressor training


In [82]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

### Train LinearRegressor


In [94]:
polyfit = False
binary = False

X = ratio_df[list(feat_map.keys())]
y = ratio_df["Overall"]

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(
    X, y, ratio_df.index, test_size=0.2, random_state=42
)

# Save the validation set
validation_set = ratio_df.loc[test_idx]
validation_set.to_json("validation_set.jsonl", lines=True, orient="records")


print(f"Train size: {len(X_train)}, test size: {len(X_test)}")

model = LinearRegression()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
# print(f"Feature names: {poly.get_feature_names_out(X.columns)}")
print(f"MSE: {mse}, RMSE: {rmse}")
print(f"Coeeficients: {model.coef_}")
print(f"Intercept: {model.intercept_}")

Train size: 101, test size: 26
MSE: 0.0011864301508304313, RMSE: 0.03444459537910746
Coeeficients: [-1.37611737e-01  5.32350462e-03 -4.38176661e-02 -2.78198558e-02
 -1.06831035e-01 -3.35084994e-03 -4.75821014e-02  4.79237520e-02
 -1.47971174e-01  2.18926230e-02  5.41783312e-03 -4.57481228e-03
 -6.07317999e-02 -6.89662029e-02 -1.08568406e-02 -2.09692174e-01
  8.62842876e-02 -6.75112570e-02  1.17200298e-02  3.68671239e-13
  1.39344092e-12 -1.00469633e-11  1.64168679e-12 -3.70847797e-12
 -9.48100706e+00 -1.83325684e-01 -2.82082828e-01 -2.87037841e-02
  3.77673356e-02 -1.27542421e-12 -1.16351373e-13 -9.55525224e-01
 -7.29455719e-03 -1.41946718e-02 -5.73052218e-01 -3.03469370e+00
 -8.85806350e-01 -2.84217094e-14 -5.48969000e-03 -5.68434189e-14
 -2.69072819e-01 -5.32907052e-15  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  2.23303094e-01  0.00000000e+00
 -6.86106181e-01  0.00000000e+00 -9.52013335e+00  0.00000000e+00
  0.00000000e+00  3.80789674e-01  3.61581246e+00  0.0000

### Train LightGBM


In [84]:
import lightgbm as lgb

,hash,model_type,chat_template,bertscore__min_val=0.33|max_val=0.67,bertscore__min_val=0.67|max_val=1.0,bertscore_length__min_val=0.0|max_val=0.33,bertscore_length__min_val=0.33|max_val=0.67,bertscore_length__min_val=0.67|max_val=1.0,complexity_of_intents=complex,complexity_of_intents=moderate,...,llmbar-natural,math-prm,mt-bench-easy,mt-bench-hard,mt-bench-med,refusals-dangerous,refusals-offensive,xstest-should-refuse,xstest-should-respond,label
26,e30383c5ba2d5942666b453a426a179e,Seq. Classifier,tulu,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.76,0.809843,0.892857,0.621622,0.875,0.46,0.35,0.525974,0.892,1
112,d4976d93b68703bb4874cbafdbb998cc,Seq. Classifier,tulu,0.000000,0.000000,0.000000,0.000000,0.209714,0.000000,0.000000,...,0.78,0.827740,0.857143,0.621622,0.775,0.18,0.25,0.344156,0.900,1
81,07ecf2a9caa729c6bd854596b88c2dbc,Seq. Classifier,tulu,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.81,0.901566,0.928571,0.567568,0.850,0.19,0.23,0.389610,0.876,1
55,5cb0c855815eadd33f80f446110f7f0f,Seq. Classifier,tulu,0.000000,0.000000,0.000000,0.000000,0.000000,0.094143,0.000000,...,0.79,0.892617,0.857143,0.594595,0.825,0.29,0.29,0.525974,0.872,1
44,de29f569e4985f516b5d4a43f62003fc,Seq. Classifier,tulu,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.75,0.868009,0.821429,0.567568,0.800,0.48,0.40,0.590909,0.864,1
96,098dc0471dde428b991422a97513168e,Seq. Classifier,tulu,0.000000,0.000000,0.425143,0.000000,0.000000,0.000000,0.000000,...,0.76,0.894855,0.857143,0.648649,0.825,0.21,0.20,0.389610,0.880,1
80,3421c61f28da40ff906276bad8240caa,Seq. Classifier,tulu,0.000000,0.967571,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.82,0.854586,0.928571,0.648649,0.800,0.30,0.15,0.318182,0.936,1
126,cd4f92195657a925382da9805f6d54f1,Seq. Classifier,tulu,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.188571,...,0.75,0.876957,0.857143,0.702703,0.775,0.15,0.10,0.233766,0.868,0
115,9ad7674524326668fd2767e9e4597be8,Seq. Classifier,tulu,0.000000,0.000000,0.000000,0.367286,0.000000,0.000000,0.000000,...,0.76,0.785235,0.892857,0.567568,0.800,0.35,0.19,0.389610,0.864,1
62,5ec871bbaf74f83bd221444addb008d6,Seq. Classifier,tulu,0.000000,0.967714,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.78,0.845638,0.785714,0.594595,0.875,0.38,0.32,0.454545,0.896,1


## Simulation


In [12]:
_, combinations = sample_feature_combinations(
    meta_analyzer_n_samples=2000, max_number=10
)

10it [00:00, 79891.50it/s]
45it [00:00, 102300.10it/s]
120it [00:00, 75800.67it/s]
210it [00:00, 65477.54it/s]
252it [00:00, 51564.28it/s]
210it [00:00, 42786.55it/s]
120it [00:00, 37153.35it/s]
45it [00:00, 28233.91it/s]
10it [00:00, 19266.44it/s]
1it [00:00, 10782.27it/s]

2024-09-03 20:42:23 - INFO - root - Adding meta analyzer features



10it [00:00, 108942.96it/s]
45it [00:00, 83923.38it/s]
120it [00:00, 78373.79it/s]
210it [00:00, 63153.64it/s]
252it [00:00, 51281.58it/s]
210it [00:00, 42083.32it/s]
120it [00:00, 19333.04it/s]
45it [00:00, 29662.69it/s]
10it [00:00, 17396.53it/s]
1it [00:00, 14122.24it/s]


In [13]:
sim_df = pd.DataFrame(0, index=np.arange(len(combinations)), columns=X.columns)
for idx, combination in tqdm_notebook(enumerate(combinations), total=len(combinations)):
    activated_feats = []
    for feat in combination:
        if "analyzer" in feat:
            feature_name_str, value_str = feat.split("::")[1].split("|")
            feature_name, value = (
                feature_name_str.split("=")[-1],
                value_str.split("=")[-1],
            )
            activated_feats.append(f"{feature_name}={value}")
        else:
            activated_feats.append(feat.replace("::", "__"))
    sim_df.loc[idx, activated_feats] = 1
sim_df = sim_df.apply(
    lambda row: row.map(lambda x: feat_map.get(row.name, 1) if x == 1 else x)
).dropna(axis=1, how="any")

/var/folders/hx/gk2rs0792pn5p8hkj4nkhdm80000gp/T/ipykernel_24031/4023816742.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, combination in tqdm_notebook(enumerate(combinations), total=len(combinations)):


  0%|          | 0/4069 [00:00<?, ?it/s]

In [14]:
sim_results = sim_df.copy(deep=True)
sim_results["activated_features"] = sim_results.apply(
    lambda row: [col for col in sim_results.columns if row[col] != 0], axis=1
)
sim_results["pred"] = model.predict(sim_df)
sim_results = sim_results.sort_values(by="pred", ascending=False).reset_index(drop=True)
sim_results[["activated_features", "pred"]].head(20)

,activated_features,pred
0,"[complexity_of_intents=simple, languages=Engli...",0.764134
1,"[complexity_of_intents=simple, languages=English]",0.760119
2,"[complexity_of_intents=simple, languages=Engli...",0.755809
3,"[complexity_of_intents=simple, cosine_sim__min...",0.753833
4,"[complexity_of_intents=simple, cosine_sim__min...",0.751735
5,[complexity_of_intents=simple],0.748626
6,[complexity_of_intents=simple],0.748626
7,[complexity_of_intents=simple],0.748626
8,[complexity_of_intents=simple],0.748626
9,[complexity_of_intents=simple],0.748626


In [26]:
top_n = 100
human_score = 0.715
better_than_humans = sim_results[sim_results["pred"] > human_score]
top_combinations = (
    better_than_humans.activated_features.head(top_n).drop_duplicates().to_list()
)
print(top_combinations)

[['complexity_of_intents=simple', 'languages=English', 'open_endedness=no', 'safety_concern=moderate'], ['complexity_of_intents=simple', 'languages=English'], ['complexity_of_intents=simple', 'languages=English', 'safety_concern=safe'], ['complexity_of_intents=simple', 'cosine_sim__min_val=0.33|max_val=0.67'], ['complexity_of_intents=simple', 'cosine_sim__min_val=0.33|max_val=0.67', 'rouge__min_val=0.0|max_val=0.33'], ['complexity_of_intents=simple'], ['bertscore__min_val=0.67|max_val=1.0', 'complexity_of_intents=simple', 'cosine_sim__min_val=0.67|max_val=1.0', 'entity_sim__min_val=0.0|max_val=0.33', 'rouge__min_val=0.33|max_val=0.67'], ['complexity_of_intents=simple', 'cosine_sim__min_val=0.67|max_val=1.0', 'languages=English', 'rouge__min_val=0.67|max_val=1.0'], ['complexity_of_intents=simple', 'format_constraints=1', 'languages=English', 'open_endedness=no', 'safety_concern=moderate'], ['complexity_of_intents=simple', 'format_constraints=1', 'languages=English', 'open_endedness=no']

So now you have determined 10 feature combinations that seem to work well. The next step is to train RMs and evaluate them.


In [28]:
from beaker import Beaker, ExperimentSpec
from copy import deepcopy

In [29]:
spec = ExperimentSpec.from_file("../../beaker/template.yml")
exp_spec = deepcopy(spec)
template_task = exp_spec.tasks.pop(0)

new_tasks = []
for idx, combination in enumerate(top_combinations):
    feats_to_run = []
    for feat in combination:
        if "min_val" in feat:
            if "token_len_difference" in feat:
                feat = feat.replace("difference", "diff")
            feats_to_run.append(feat.replace("__", "::"))
        else:
            feat_name, value = feat.split("=")
            category = find_meta_category(feat_name)
            if category == "closed_set":
                key = "constraints"
            elif category == "scalar":
                key = "value"
            elif category == "open_set":
                key = "check_for_existence"
            feats_to_run.append(f"{category}::feature_name={feat_name}|{key}={value}")
    # Create beaker task
    task = deepcopy(template_task)
    task.name = f"get-features-datamodel-{idx}"
    task.arguments.extend(["--features"] + feats_to_run)
    new_tasks.append(task)

exp_spec.tasks = new_tasks
exp_spec.validate()
exp_spec.to_file("experiments.yml")

Get finished jobs and download the subsets and create an `experiments.txt` file


In [30]:
# experiment_id = "01J6TS47Q2KNKYRCYHC8A0DE4B"
# experiment_id = "01J6WDKDPQCM92REXJ1VCNJ0NW"
experiment_id = "01J6XJSWMSAM2ARJ1WXAP6PV8T"
top_subsets_dir = Path("top_n_subsets")
top_subsets_dir.mkdir(parents=True, exist_ok=True)
experiments_file = top_subsets_dir / "top_n_subsets_experiments.txt"
beaker = Beaker.from_env("ai2/ljm-oe-adapt")
experiment = beaker.experiment.get(experiment_id)

experiment_ids = []
for job in tqdm_notebook(experiment.jobs):
    if job.is_done:
        # Get output
        dataset_id = job.execution.result.beaker
        beaker.dataset.fetch(
            dataset_id,
            force=True,
            target=top_subsets_dir,
            prefix="data/",
            quiet=True,
        )

        beaker.dataset.fetch(
            dataset_id,
            force=True,
            target=top_subsets_dir,
            prefix="experiments.txt",
            quiet=True,
        )

        with open(top_subsets_dir / "experiments.txt", "r") as f:
            data = f.read().splitlines()
            id = data[0]
            experiment_ids.append(id)

print(experiment_ids)
with open(experiments_file, "a") as f:
    for id in set(experiment_ids):
        f.write("\n" + id)

/var/folders/hx/gk2rs0792pn5p8hkj4nkhdm80000gp/T/ipykernel_24031/76539003.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for job in tqdm_notebook(experiment.jobs):


  0%|          | 0/65 [00:00<?, ?it/s]

['human_datamodel_7000_FEATS_818fcd7dfafaf799a556f7f3363e7765_SWAPS_1922::bertscore__min_val-0.67|max_val-1.0___scalar__feature_name-complexity_of_intents|value-simple___entity_sim__min_val-0.67|max_val-1.0___scalar__feature_name-safety_concern|value-safe', 'human_datamodel_7000_FEATS_818fcd7dfafaf799a556f7f3363e7765_SWAPS_1922::bertscore__min_val-0.67|max_val-1.0___scalar__feature_name-complexity_of_intents|value-simple___entity_sim__min_val-0.67|max_val-1.0___scalar__feature_name-safety_concern|value-safe', 'human_datamodel_7000_FEATS_818fcd7dfafaf799a556f7f3363e7765_SWAPS_1922::bertscore__min_val-0.67|max_val-1.0___scalar__feature_name-complexity_of_intents|value-simple___entity_sim__min_val-0.67|max_val-1.0___scalar__feature_name-safety_concern|value-safe', 'human_datamodel_7000_FEATS_4bf313ac33a2c5007338baf8ef25ec52_SWAPS_1658::scalar__feature_name-complexity_of_intents|value-simple___cosine_sim__min_val-0.33|max_val-0.67', 'human_datamodel_7000_FEATS_4bf313ac33a2c5007338baf8ef25e

In [19]:
len(set(experiment_ids))

13